In [8]:
import chromadb
import os
import json
# Uso questa libreria per cambiare LLM
from chromadb.utils import embedding_functions
chroma_client = chromadb.HttpClient(host="localhost", port=8000)

In [ ]:
# Vado a recuperare dal file topic i topic, in modo da inserirli nel database 
with open('data/topic.json', 'r') as f:
    topic_data = json.load(f)

documents=[]
metadatas=[]
ids=[]
id=1


for topic in topic_data:
    for top in topic_data[topic]:
        metadatas.append({"source":"topic.json"})
        documents.append(top)
        ids.append(str(id))
        id+=1

In [19]:
# Cambio LLM usato-> di base è "all-MiniLM-L6-v2"
sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-mpnet-base-v2")
# Crea database -> gli comunico llm che voglio
collection = chroma_client.create_collection(name="my_collection", embedding_function=sentence_transformer_ef)
#collection = chroma_client.create_collection(name="my_collection")

Exception: {"error":"UniqueConstraintError('Collection my_collection already exists')"}

In [11]:
#sto anddando ad insesire 2 nuovi elementi con il rispettivo document, source ed id
collection.add(
    documents=documents,
    metadatas=metadatas,
    ids=ids
)

In [18]:
# utilizzo la query per recuperarlo, tentendo conto che non sto andando a cercare precisamente
# quello ceh c'è scritto in un documenti, quindi verrà calcolata una sorta di distanza  
results = collection.query(
    # questa è la query che utilizzerò in base alle operazion iche devo svolgere
    query_texts=["Racconti di avventure nel mondo della magia"],
    n_results=5,
    # Scelgo i campi che voglio mi vengano tornati
    include=['distances','metadatas','documents']
)
# la query restituirà una distance che più è piccola più il contenuto che sto cercando sarà simile a quello comaprato
results

{'ids': [['96', '19', '75', '61', '98']],
 'distances': [[0.7343766689300537,
   0.7370734810829163,
   0.7660244107246399,
   0.8093665838241577,
   0.8215274810791016]],
 'embeddings': None,
 'metadatas': [[{'source': 'topic.json'},
   {'source': 'topic.json'},
   {'source': 'topic.json'},
   {'source': 'topic.json'},
   {'source': 'topic.json'}]],
 'documents': [['Consigli di relazione',
   'Disegno e illustrazione',
   'Giochi per dispositivi mobili',
   'Lavorazione del legno',
   'Consigli per appuntamenti']],
 'uris': None,
 'data': None}